# MNIST DP-SGD Keras

## Imports and Google Drive

In [1]:
"""Evaluate the DP-SGD optimizer using TF 1.x.

Code of the Notebook is based on
https://github.com/tensorflow/privacy/blob/master/tutorials/
mnist_dpsgd_tutorial_keras.py. For a quick walktrough see
https://github.com/tensorflow/privacy/tree/master/tutorials/walkthrough.

Attributes
----------
GradientDescentOptimizer : tf.train.GradientDescentOptimizer
    Non-DP optimizer for the training.

License of the code underlying this notebook
--------------------------------------------
Copyright 2019, The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
from timeit import default_timer as timer

# set tensorlfow version in google colab
try:
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow.compat.v1 as tf

# used to measure the privacy gurantee
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import \
    get_privacy_spent

# optimizer used for the privacy-preserving training
from tensorflow_privacy.privacy.optimizers.dp_optimizer import \
    DPGradientDescentGaussianOptimizer

GradientDescentOptimizer = tf.train.GradientDescentOptimizer

# mount google drive
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

## Compute Epsilon Values

In [2]:
def compute_epsilon(noise_multiplier, minibatches, epochs):
  """Computes epsilon value for given hyperparameters.

      Epsilon describes the strength of our privacy guarantee. In the case of
      DP-ML, it gives a bound on how much the probability of a particular model
      output can vary by including (or removing) a single training example. We
      usually want it to be a small constant. However, this is only an upper
      bound, and a large value of epsilon could still mean good practical
      privacy. Interpreting this value could be quiet difficult.

  Returns
  -------
  float
      Epsion-value for the expanded privacy budget.

  Parameters
  ----------
  noise_multiplier : float
      Ratio of the standard deviation to the clipping norm. Typically more
      noise results in stronger privacy and often at the expense of utility.
  minibatches : int
      Number of samples used in each training step.
  epochs : int
      Number of training iterations.
  """
  # Together with the noise multiplier are these the parameters which are
  # relevant to measuring the potential privacy loss induced by the training.
  #
  # * sampling_probability: The probability of an individual training point
  # being included in a minibatch.
  # * steps: Number of steps the optimizer takes over the training data.
  steps = epochs * 60000 // minibatches
  sampling_probability = minibatches / 60000

  if noise_multiplier == 0.0:
    return float('inf')

  # TODO: Understand the exact meaning of the orders parameter
  # The exact meaning of the orders is not quite clear to me yet. For a rough
  # description and a rule of thumb see the walkthrough.
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))

  # Rule of thump: Delta is set to 1e-5 because MNIST has 60000 training
  # points (see the walktrough). Delta bounds the probability that our privacy
  # guarantee do not hold.
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=steps,
                    orders=orders)

  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

## Load and preprocess MNIST Dataset

In [3]:
def load_mnist():
  """Loads MNIST and preprocesses to combine training and validation data.

  Returns
  -------
  tuple
      (history of the training, runtime measurement, epsilon-value)
  """
  train, test = tf.keras.datasets.mnist.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
  test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  assert train_data.min() == 0.
  assert train_data.max() == 1.
  assert test_data.min() == 0.
  assert test_data.max() == 1.

  return train_data, train_labels, test_data, test_labels

## Design Model Architecture

In [4]:
def create_model():
  from tensorflow.keras import Sequential
  from tensorflow.keras.layers import Conv2D
  from tensorflow.keras.layers import MaxPool2D
  from tensorflow.keras.layers import Flatten
  from tensorflow.keras.layers import Dense
  
  model = Sequential()
  model.add(Conv2D(16, 8, strides=2, padding='same', activation='relu', 
                   input_shape=(28, 28, 1)))
  model.add(MaxPool2D(2, 1))
  model.add(Conv2D(32, 4, strides=2, padding='valid', activation='relu'))
  model.add(MaxPool2D(2, 1))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(10))
  
  return model

## Train Model: SGD or DP-SGD

In [5]:
def main(dpsgd, learning_rate, noise_multiplier, l2_norm_clip, minibatches,
         epochs, microbatches, callbacks=None, verbose=1):
  """Define, train and compute the used privacy budget of the keras model.

  Raises
  ------
  ValueError
      The number of microbatches must divide the batch size.

  Parameters
  ----------
  dpsgd : bool
      If True, train with DP-SGD. If False, train with vanilla SGD.
  learning_rate : float
      Learning rate for training.
  noise_multiplier : float
      Ratio of the standard deviation to the clipping norm. Typically more
      noise results in stronger privacy and often at the expense of utility.
  l2_norm_clip : float
      Attribute gives the maximum Euclidean norm of each individual gradient
      that is computed on an individual training example from a minibatch. This
      parameter is used to bound the optimizer's sensitivity to individual
      training points.
  minibatches : int
      Number of samples used in each training step.
  epochs : int
      Number of epochs used for the training.
  microbatches : int
      Number of microbatches (must be evently divide batch size). In practice
      clipping gradients for each exampe indivdudally can strongly degrade the
      performance because instead of parallelizing at the granularity of
      minibatches the computations must be performed for each example. Rather
      than clipping gradients per example we clip them on the basis of
      microbatches. In this way is the number of microbatches a trade-off
      parameter between privacy and utility (small number -> higher privacy,
      number closer to size of minibatches -> higher utility).
  verbose : int, optional
      Verbose parameter of the TF fit function.

  Returns
  -------
  tuple
      Training history, training time, epsilon value (see compute_epsilon())
  """
  if dpsgd and minibatches % microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly minibatches')

  # Load training and test data.
  train_data, train_labels, test_data, test_labels = load_mnist()

  # Define a sequential Keras model
  model = create_model()

  if dpsgd:
    optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=microbatches,
        learning_rate=learning_rate)
    # Compute vector of per-example loss rather than its mean over a minibatch.
    # The optimizers needs the loss per example in order to compute the
    # gradients per example (rather than per minibatch) and clip/noise the
    # gradient of each example individually.
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
  else:
    optimizer = GradientDescentOptimizer(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

  # Compile model with Keras
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  # Train model with Keras and measure training time
  start = timer()
  history = model.fit(train_data[:1000], train_labels[:1000],
                      epochs=epochs,
                      validation_data=(test_data[:1000], test_labels[:1000]),
                      batch_size=minibatches,
                      verbose=verbose,
                      callbacks=callbacks)
  end = timer()

  training_time = end - start  # time in seconds

  # Compute the privacy budget expended
  epsilon = -42
  if dpsgd:
    epsilon = compute_epsilon(noise_multiplier, minibatches, epochs)

  return history, training_time, epsilon

## Lets just fit a model

In [6]:
import time
class TimeHistory(tf.keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
      self.times = []

  def on_epoch_begin(self, batch, logs={}):
      self.epoch_time_start = time.time()

  def on_epoch_end(self, batch, logs={}):
      self.times.append(time.time() - self.epoch_time_start)

# --- Hyperparamater
dpsgd = True
learning_rate = 0.15
noise_multiplier = 1.1
l2_norm_clip = 1.0
# For the DP Optimizer the size of the minibatches must divide the number of
# training samples!
minibatches = 125
epochs = 2
microbatches = 125
verbose=1

# --- Callbacks
time_callback = TimeHistory()
callbacks = [time_callback]

# tensorboard callback does not work with dp optimizer, but you can use this
# code to optimize non-dp models
#if !dpsgd:
#  board_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
#                                                  histogram_freq=1,
#                                                  update_freq='epoch',
#                                                  profile_batch=0)
#  callbacks.append(board_callback)
#  %tensorboard --logdir logs/fit

In [7]:
# Train the keras model and compute the used privacy budget.
_, training_time, epsilon = main(dpsgd, learning_rate, noise_multiplier,
                                 l2_norm_clip, minibatches, epochs,
                                 microbatches, callbacks)

print('Training time: ', training_time)
if dpsgd:
  print('For delta=1e-5, the current epsilon is: %.2f' % epsilon)
else:
  print('Trained with vanilla non-private SGD optimizer')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1000 samples, validate on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 5s 5ms/sample - loss: 2.3011 - acc: 0.1460 - val_loss: 2.2860 - val_acc: 0.1640
Epoch 2/2
1000/1000 [==============================] - 3s 3ms/sample - loss: 2.2730 - acc: 0.1900 - val_loss: 2.2631 - val_acc: 0.2270
Training time:  9.018492429051548
For delta=1e-5, the current epsilon is: 0.93


In [11]:
print(time_callback.times)

[4.984344959259033, 2.5537402629852295]


## Evaluate DP-SGD

In [ ]:
# We evaluate the DP-SGD with the following different
# hyperparameters in terms of epsilon, accuracy and running time. We run each
# each hyperparameter configuration (run) in a loop of 3 iterations.

dps = [False, True, True, True]
learning_rates = [0.1, 0.25, 0.15, 0.25]
noise_multipliers = [1.3, 1.1, 0.7]
clipping_thresholds = [1.5, 1, 1.5]
minibatches = 250
microbatches = 250
epochs = [20, 15, 60, 45]

# a running index
dp_idx = -1
# save the hyperparameters of each iteration in a list
rows = []
for idx, dp in enumerate(dps):
  if dp:
    dp_idx += 1

  for i in range(3):
    print('Run: ', idx, ' - Loop: ', i)

    history, training_time, epsilon = main(dp, learning_rates[idx],
                                           noise_multipliers[dp_idx],
                                           clipping_thresholds[dp_idx],
                                           minibatches, epochs[idx],
                                           microbatches, 0)
  
    accuracy = history.history['val_acc'][-1]
  
    # save the hyperparameters per row in a dict
    row = {'dp': dp,
           'learning_rate': learning_rates[idx],
           'noise_multiplier': noise_multipliers[dp_idx],
           'clipping_threshols': clipping_thresholds[dp_idx],
           'minibatches': minibatches,
           'microbatches': microbatches,
           'epochs': epochs[idx],
           'epsilon': epsilon,
           'accuracy': accuracy,
           'training_time': training_time}

    rows.append(row)

df = pd.DataFrame(rows)

In [8]:
# save/load and clean up df
#df.to_pickle('/content/drive/My Drive/df.pkl')
df = pd.read_pickle('/content/drive/My Drive/df.pkl')
df.loc[df.dp==False, ['epsilon', 'noise_multiplier', 'clipping_threshols']] = np.nan
df['Run'] = np.array([[i]*3 for i in range(4)]).flatten()
df = df.set_index('Run')
df.loc[:, 'training_time'] /= 60
df

,dp,learning_rate,noise_multiplier,clipping_threshols,minibatches,microbatches,epochs,epsilon,accuracy,training_time
Run,,,,,,,,,,
0,False,0.10,NaN,NaN,250,250,20,NaN,0.9896,0.537926
0,False,0.10,NaN,NaN,250,250,20,NaN,0.9896,0.313726
0,False,0.10,NaN,NaN,250,250,20,NaN,0.9908,0.312303
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.9506,17.198169
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.9473,17.125886
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.9509,17.115841
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.9679,68.369128
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.9639,68.144310
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.9656,68.915586


In [9]:
# calculate the relative standard derivation
(df.loc[:,['accuracy', 'training_time']].std(level='Run')/
df.loc[:,['accuracy', 'training_time']].mean(level='Run'))

,accuracy,training_time
Run,,
0,0.000700,0.334690
1,0.002104,0.002619
2,0.002079,0.005793
3,0.001083,0.016797


In [10]:
# calculate the mean of each run
df = df.mean(level='Run')
df

,dp,learning_rate,noise_multiplier,clipping_threshols,minibatches,microbatches,epochs,epsilon,accuracy,training_time
Run,,,,,,,,,,
0,False,0.10,NaN,NaN,250,250,20,NaN,0.990000,0.387985
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.949600,17.146632
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.965800,68.476342
3,True,0.25,0.7,1.5,250,250,45,7.009134,0.969533,53.398623


In [11]:
df = df.drop(labels=['minibatches', 'microbatches'], axis=1)
print(df.to_latex(index=False))

\begin{tabular}{lrrrrrrr}
\toprule
    dp &  learning\_rate &  noise\_multiplier &  clipping\_threshols &  epochs &   epsilon &  accuracy &  training\_time \\
 False &           0.10 &               NaN &                 NaN &      20 &       NaN &  0.990000 &       0.387985 \\
\midrule
  True &           0.25 &               1.3 &                 1.5 &      15 &  1.179901 &  0.949600 &      17.146632 \\
  True &           0.15 &               1.1 &                 1.0 &      60 &  2.969930 &  0.965800 &      68.476342 \\
  True &           0.25 &               0.7 &                 1.5 &      45 &  7.009134 &  0.969533 &      53.398623 \\
\bottomrule
\end{tabular}

